In [1]:
from requests import get as r_get
from json import loads
from csv import DictWriter
from schemas import Vacancies, Vacancy

In [2]:
def get_vacancy(id: int) -> Vacancy:
    """_summary_

    Args:
        id (int): _description_

    Returns:
        Vacancy: _description_
    """
    with r_get(f"https://api.hh.ru/vacancies/{id}") as req:
        data = loads(req.content.decode())
        print(f"Вакансия добавлена: {data.get('name')}")
        v_data = Vacancy.parse_obj(data)
        return v_data

In [3]:
def get_initial_data() -> Vacancies:
    params = {
        "text": "NAME:Backend",  # NAME:Backend Текст фильтра. В имени должно быть слово "Backend"
        "area": 113,  # Поиск ощуществляется по вакансиям России
        "page": 0,  # Индекс страницы поиска на HH
        "per_page": 100,  # Кол-во вакансий на 1 странице
    }
    with r_get("https://api.hh.ru/vacancies", params) as req:
        return Vacancies.parse_obj(loads(req.content.decode()))

In [4]:
def getPage(page: int = 0) -> list[Vacancy]:
    """
    Создаем метод для получения страницы со списком вакансий.

    Args:
        page (int, optional): Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница

    Returns:
        str: вакансии
    """
    params = {
        "text": "NAME:Backend",  # NAME:Backend Текст фильтра. В имени должно быть слово "Backend"
        "area": 113,  # Поиск ощуществляется по вакансиям России
        "page": page,  # Индекс страницы поиска на HH
        "per_page": 100,  # Кол-во вакансий на 1 странице
    }
    with r_get("https://api.hh.ru/vacancies", params) as req:  # Посылаем запрос к API
        data = loads(req.content.decode()).get("items", [])
    return [get_vacancy(item.get("id")) for item in data]

In [5]:
all_vacancies = []
first_page = get_initial_data()
pages = first_page.pages
for page in range(1, pages):
    all_vacancies += getPage(page)
    print(f"Вакансии: {page}/{pages} ({len(all_vacancies)})")

Вакансия добавлена: Backend программист, Erlang
Вакансия добавлена: Python Developer (Backend)
Вакансия добавлена: Backend developer (Бэкенд разработчик)
Вакансия добавлена: Ruby on Rails Backend разработчик
Вакансия добавлена: Backend-разработчик
Вакансия добавлена: Backend-разработчик
Вакансия добавлена: Backend разработчик
Вакансия добавлена: Python backend developer
Вакансия добавлена: Senior .Net Backend Developer
Вакансия добавлена: Backend-разработчик
Вакансия добавлена: Backend - разработчик
Вакансия добавлена: Backend разработчик
Вакансия добавлена: .Net Backend Developer
Вакансия добавлена: Backend Java разработчик
Вакансия добавлена: Middle/Senior Backend Developer (Node.js)
Вакансия добавлена: Backend developer (Python, Go)
Вакансия добавлена: Разработчик Backend (Java/Kotlin) senior
Вакансия добавлена: Backend Developer с опытом разработки на Node.js. Middle +/Senior.
Вакансия добавлена: Node.js Backend Developer
Вакансия добавлена: Backend разработчик (PHP, Laravel, MySQL

In [6]:
fieldnames = list(all_vacancies[0].schema()["properties"].keys())

In [7]:
with open("vacancies.csv", "w", newline="") as output_file:
    fieldnames = list(all_vacancies[0].schema()["properties"].keys())
    dict_writer = DictWriter(output_file, fieldnames)
    dict_writer.writeheader()
    for vacancy in all_vacancies:
        vacancy = vacancy.dict()
        if vacancy.get("salary"):
            from_ = vacancy.get("salary").get("start") or 0
            to_ = vacancy.get("salary").get("to") or 0
            print(from_, to_, vacancy.get("salary"))
            average_salary = (from_ + to_) // 2
            if from_ == 0:
                average_salary = to_
            if to_ == 0:
                average_salary = from_
            vacancy["salary"] = average_salary if vacancy.get("salary").get("currency") == "RUR" else average_salary * 80
        vacancy["experience"] = vacancy.get("experience").get("name")
        vacancy["area"] = vacancy.get("area").get("name")
        vacancy["key_skills"] = ", ".join([skill.get("name") for skill in vacancy["key_skills"]])
        dict_writer.writerow(vacancy)

200000 0 {'start': 200000, 'to': None, 'currency': 'RUR', 'gross': False}
150000 190000 {'start': 150000, 'to': 190000, 'currency': 'RUR', 'gross': False}
100000 0 {'start': 100000, 'to': None, 'currency': 'RUR', 'gross': False}
55000 91000 {'start': 55000, 'to': 91000, 'currency': 'RUR', 'gross': False}
0 90000 {'start': None, 'to': 90000, 'currency': 'RUR', 'gross': True}
0 500000 {'start': None, 'to': 500000, 'currency': 'RUR', 'gross': False}
200000 300000 {'start': 200000, 'to': 300000, 'currency': 'RUR', 'gross': False}
70000 100000 {'start': 70000, 'to': 100000, 'currency': 'RUR', 'gross': True}
140000 200000 {'start': 140000, 'to': 200000, 'currency': 'RUR', 'gross': False}
160000 250000 {'start': 160000, 'to': 250000, 'currency': 'RUR', 'gross': False}
0 300000 {'start': None, 'to': 300000, 'currency': 'RUR', 'gross': True}
2500 4000 {'start': 2500, 'to': 4000, 'currency': 'USD', 'gross': False}
0 330000 {'start': None, 'to': 330000, 'currency': 'RUR', 'gross': False}
4000 600